<!-- Table des matières automatique -->
<div style="overflow-y: auto">
  <h1>Table des matières</h1>
  <div id="toc"></div>
</div>

In [ ]:
%%javascript
$.getScript('ipython_notebook_toc.js')

# ACDC 2019-2020 : l'Autre Carré D'à Côté

Mise au propre des données et analyses stats

In [ ]:
%matplotlib inline

In [ ]:
import sys
import os
import math
import re
import datetime as dt
import pandas as pd
import numpy as np

from lxml import etree
import pyproj

import folium
import folium.plugins

import simplekml as skml # Simple KML generation API
import kmlcircle # Circle generation for KML generation

from shapely import geometry

from matplotlib import pyplot as plt
import matplotlib.gridspec as pltg

from IPython.display import HTML

# Communs

In [ ]:
KInfValues = [np.inf, -np.inf]

In [ ]:
kmlNameSpaces = \
  { 'gx' : 'http://www.google.com/kml/ext/2.2',
    'kml' : 'http://www.opengis.net/kml/2.2',
    'atom' : 'http://www.w3.org/2005/Atom' }

In [ ]:
# Projection coordonnées sphériques (degrés)=> coordonnées planes (système cible au choix)
KProjWgs84 = pyproj.Proj(init='epsg:4326') # WGS 84 : long, lat en degrés

KProjUtm31  = pyproj.Proj(init='epsg:32631') # WGS 84 - UTM 31N : long, lat en m
KProjCc46   = pyproj.Proj(init='epsg:3946')  # RGF 93 - CC46    : long, lat en m
KProjLamb93 = pyproj.Proj(init='epsg:2154')  # RGF 93 - Lambert : long, lat en m

def geoProjeter(sCoords, srcProj, tgtProj): # sCoords : [0]=x=long, [1]=y=lat
    return pd.Series(pyproj.transform(srcProj, tgtProj, sCoords[0], sCoords[1]))

# Stats simples des 2 passages Naturalist

## 1. Chargement des exports individuels Faune Auvergne

(exports par observateur, passage 'a' du 1er avril au 15 mai, 'b' du 16 mai au 15 juin)

In [ ]:
# Chargement des exports
dfObsBrutes = pd.DataFrame()
iFic = 1
for nomFic in os.listdir('ACDC'):
    mo = re.match('ACDC2019([ab])-Export(.*).xlsx', nomFic)
    if mo and '.old' not in nomFic:
        dfObsIndiv = pd.read_excel(os.path.join('ACDC', nomFic))
        dfObsIndiv['Passage'] = mo.group(1)
        dfObsBrutes = dfObsBrutes.append(dfObsIndiv, ignore_index=True, sort=False)
        print('#{} ACDC/{} : {} données brutes, total {}'.format(iFic, nomFic, len(dfObsIndiv), len(dfObsBrutes)))
        iFic += 1

#dfObsBrutes.DATE.fillna(value=pd.Timestamp('{}-{}'.format(annee, dateDefaut)), inplace=True) # Date par défaut
dfObsBrutes['Observateur'] = dfObsBrutes[['Nom', 'Prénom']].apply(lambda s: s[1]+' '+s[0], axis='columns')

len(dfObsBrutes)

In [ ]:
dfObsBrutes.columns

In [ ]:
dfObsBrutes.tail().T

In [ ]:
# Parenthèse : Comparaison des exports FA SIMPLE et JPM+ :
#dfExSimp = pd.read_excel(os.path.join('ACDC', 'export_29062019_122605.xlsx'))
#dfExSimp = pd.DataFrame(data=dict(exportSimple=dfExSimp.columns)).set_index('exportSimple', drop=False)
#dfExJpmP = dfObsBrutes.iloc[:1].drop(['Observateur', 'Passage'], axis='columns'])
#dfExJpmP = pd.DataFrame(data=dict(exportJpmPlus=dfObsBrutes.columns)).set_index('exportJpmPlus', drop=False)
#dfExComp = dfExSimp.join(dfExJpmP, how='outer').reset_index(drop=True)

In [ ]:
# a. les colonnes de JPM+ qui ne sont pas dans SIMPLE
#dfExComp[dfExComp.exportSimple.isnull()]

In [ ]:
# b. les colonnes de SIMPLE qui ne sont pas dans JPM+
#dfExComp[dfExComp.exportJpmPlus.isnull()]

In [ ]:
# Sélection des colonnes utiles
colBrutes = ['Ref', 'ID liste', 'Liste complète ?', 'Commentaire de la liste',
             'Passage', 'Date', 'Heure début', 'Heure fin', 'Horaire',
             'Ordre systématique', 'Nom espèce', 'Estimation', 'Nombre', 'Détails',
             'Code atlas', 'Lat (WGS84)', 'Lon (WGS84)', 'Altitude',
             'Lieu-dit', 'Commune', 'Remarque', 'Remarque privée', 'Observateur']
dfObs = dfObsBrutes[colBrutes]

In [ ]:
dfObs.tail()

In [ ]:
# Examen des formulaires, pour pister ceux qui nous intéressent, en soupçonnant des erreur de saisie du numéro de point
df = dfObs.loc[(dfObs['ID liste'] > 0)]
len(df), df['ID liste'].nunique(), ', '.join(sorted(df['Commune'].unique()))

In [ ]:
# Les communes concernées (à qq données près probablement)
df = df[(df.Commune.isin(['Ludesse', 'Cournols', 'Olloix', 'Montaigut-le-Blanc']))]
len(df), df['ID liste'].nunique()

In [ ]:
df = df[['ID liste', 'Date', 'Heure début', 'Heure fin', 'Commentaire de la liste', 'Observateur', 'Lieu-dit', 'Commune']]

In [ ]:
dff = df.groupby(['ID liste']).first()

In [ ]:
list(dff['Commentaire de la liste'].unique())

In [ ]:
# Encore des formulaires ACDC sans le commentaire ACDC ?
dff[~dff['Commentaire de la liste'].fillna('').str.contains('acdc', case=False)]

# 2. Filtrage des données : formulaires ACDC

In [ ]:
dfObs = dfObs[(dfObs['ID liste'] != 0) & dfObs['Commentaire de la liste'].str.contains('acdc', case=False)]

len(dfObs)

In [ ]:
dfObs.tail()

In [ ]:
# dfObs['Commentaire de la liste'].unique()

In [ ]:
#mo = re.match('acdc\s*(?:2019\s*){0,1}(?:point\s*){0,1}(\d{2,3})\D*', 'acdc  232 Google', flags=re.IGNORECASE)
#np.nan if not mo else float(mo.group(1))

# 3. Récupération du numéro de point

(dans le commentaire de chaque formulaire)

In [ ]:
(dfObs['Commentaire de la liste'].nunique(), dfObs['Commentaire de la liste'].unique())

In [ ]:
# Nettoyage préalable commentaire liste
KCommList2Rem = { '2019': '', 'point': '', 'pt':' ', 'passage': '',
                 '1er': '', ' 2 pt': '', ' 2 ': ' ', '2ème': '', '2eme': '', 'second': '',
                  '.': ' ', ':': ' ', ',': ' ', '\n': ' ', '/': ' ' }

def cleanupCommListe(commentaire):
    comm = commentaire.lower()
    for k, v in KCommList2Rem.items():
        comm = comm.replace(k, v)
    return comm.strip()

dfObs['Commentaire de la liste'] = dfObs['Commentaire de la liste'].apply(cleanupCommListe)

(dfObs['Commentaire de la liste'].nunique(), dfObs['Commentaire de la liste'].unique())

In [ ]:
KReCommListeNumPt = ['acdc\s+(\d{2,3})\s*.*', '(\d{2,3})\s+acdc\s*.*']

def numeroPointAcdc(commentaire):
    for regExp in KReCommListeNumPt:
        mo = re.match(regExp, commentaire, flags=re.IGNORECASE)
        if mo:
            break
    return np.nan if not mo else float(mo.group(1))

dfObs['Num point ACDC'] = dfObs['Commentaire de la liste'].apply(numeroPointAcdc)

In [ ]:
#numeroPointAcdc('ACDC Point 202. observateur au point.')

In [ ]:
# Nombre et liste des points effectués au moins 1 fois.
dfObs['Num point ACDC'].nunique(), dfObs['Num point ACDC'].unique()

In [ ]:
# Nombre de points sans numéro retrouvé
dfObs[dfObs['Num point ACDC'].isnull()].groupby('Commentaire de la liste').first()

In [ ]:
# Première idée des nbres de données par point et par passage (et des points effectués à chaque passage)
dfObs[['Num point ACDC', 'Passage', 'Commentaire de la liste']].groupby(['Num point ACDC', 'Passage']).count().unstack()

In [ ]:
# Inventaires de Clément, avec vraiment beaucoup de données
dfObs.loc[dfObs['Observateur'] == 'Clément Rollant', ['Date', 'Heure début', 'Nombre']].groupby(['Date', 'Heure début']).size()

In [ ]:
#dfObs.loc[dfObs['Observateur'] == 'Clément Rollant', ['Passage', 'Heure début', 'Nom espèce', 'Nombre']] \
#     .groupby(['Passage', 'Heure début', 'Nom espèce']).sum()

# 4. Suppression des inventaires en trop par passage

(bon, c'est en contrôlant les formulaires en 5. qu'on sait qu'il faut le faire :-)

In [ ]:
len(dfObs)

In [ ]:
# Romain, passage a, points 178 et 179 effectués le 14/4 et le 1er mai : on garde le meilleur passage (à meilleure heure)
sObsAVirer = dfObs['Num point ACDC'].isin([178, 179]) & (dfObs.Date == '2019-04-14')
sObsAVirer.sum()

In [ ]:
dfObs[sObsAVirer].index

In [ ]:
dfObs.drop(labels=dfObs[sObsAVirer].index, inplace=True)
len(dfObs)

# 5. Contrôle des formulaires

* numéros de point tous bien récupérés (du commentaire liste) ?
* numéros de points récupérés correspondant bien au vrais points inventoriés (cartographie) ?
* points attribués bien inventoriés à chaque passage ?
* 1 seul inventaire par passage ?
* pas de formulaire de plus de 11mn ? (normalement ...)

In [ ]:
dfObs.head()

In [ ]:
# Nombre de points sans numéro retrouvé
dfObs[dfObs['Num point ACDC'].isnull()].groupby('Commentaire de la liste').first()

In [ ]:
# On va devoir savoir si une obs est dans les 5 1ères minues ou pas
dfObs['Heure début'] = dfObs.apply(lambda s: dt.datetime.combine(s['Date'].date(), s['Heure début']), axis='columns')
dfObs['Heure fin'] = dfObs.apply(lambda s: dt.datetime.combine(s['Date'].date(), s['Heure fin']), axis='columns')
dfObs['Horaire'] = dfObs.apply(lambda s: dt.datetime.combine(s['Date'].date(), dt.time.fromisoformat(s['Horaire'])),
                               axis='columns')
dfObs['Minute'] = dfObs['Horaire'] - dfObs['Heure début']

In [ ]:
# Données hors période des formulaires (bizarre, localisées uniquement chez Romain)
#df = dfObs[(dfObs['Horaire'] < dfObs['Heure début']) | (dfObs['Horaire'] > dfObs['Heure fin'])]
#df.to_excel('ACDC2019-DonneesHoraireHorsListe.xlsx', index=False)
#df

In [ ]:
# Tous les formulaires ACDC x passages retenus
dfForms = dfObs[['ID liste', 'Passage', 'Date', 'Heure début', 'Heure fin', 'Num point ACDC', 'Observateur',
                 'Commentaire de la liste', 'Liste complète ?']] \
            .groupby(['ID liste', 'Passage', 'Date', 'Commentaire de la liste']).first()
dfForms['Durée'] = dfForms['Heure fin'] - dfForms['Heure début']
dfForms = dfForms.reset_index().set_index('ID liste').sort_values(by='Observateur')
len(dfForms)

In [ ]:
# Normalement, tous les formulaires ont duré au moins 10mn (à qq s près => >= 9 mn)
dfForms[dfForms['Durée'] < pd.Timedelta(minutes=9)]

In [ ]:
# Normalement, tous les formulaires sont des listes complètes.
dfForms[dfForms['Liste complète ?'] != 1]

In [ ]:
dfForms.drop(columns=['Liste complète ?'], inplace=True)

In [ ]:
dfForms.tail()

In [ ]:
# Numéros de points correctement attribués aux formulaires ?
# 1 seul inventaire par passage et par point ?
dfForms[dfForms.duplicated(subset=['Passage', 'Num point ACDC'], keep=False)]

In [ ]:
# Les doublons de Romain ...
#dfObs.loc[dfObs['Num point ACDC'].isin([178, 179]),
#          ['ID liste', 'Num point ACDC', 'Passage', 'Date', 'Heure début', 'Observateur']] \
#     .groupby(['ID liste', 'Num point ACDC', 'Date', 'Heure début', 'Passage']).count() \
#     .rename(columns=dict(Observateur='Nb données'))

In [ ]:
# Cartographie des données pour vérifier ça ...
obseur = 'Romain Riols'
dfObs2Map = dfObs[dfObs['Observateur'] == obseur]
dfPts2Map = dfPoints[dfPoints.naturalist == obseur]

In [ ]:
#tiles, attr = 'http://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'osm'# OK
tiles, attr = 'https://{s}.tile.thunderforest.com/outdoors/{z}/{x}/{y}.png', 'thunderforest' # OK
#tiles, attr = 'http://{s}.tile.openstreetmap.fr/fradm/{z}/{x}/{y}.png', 'osm fr' # marche pô
#tiles, attr = 'https://{s}.tile.openstreetmap.fr/qa/{zoom}/{x}/{y}.png', 'osm fr' # marche pô
mp = folium.Map(tiles=tiles, attr=attr)

# Polygone limitant à la zone d'étude
poly = folium.PolyLine(locations=[(lat, long) for long, lat in dfZonePoly[['long', 'lat']].itertuples(index=False)],
                       color='red', opacity=0.8, popup='Zone ACDC Cournols-Olloix JPD')
poly.add_to(mp)

# Les points d'écoute
for indPt, sPt in dfPts2Map.iterrows():
    mrk = folium.Marker(location=(sPt.latitude, sPt.longitude), 
                        popup=folium.Popup('#{} {}'.format(sPt.name, sPt.naturalist or 'Personne')),
                        icon=folium.Icon(color='white', icon_color='black'))
    mrk.add_to(mp)

# Les données, rassemblées par formulaire / point
for formId in dfObs2Map['ID liste'].unique():
    dfObsForm = dfObs2Map[dfObs2Map['ID liste'] == formId]
    numPt = dfObsForm.iloc[0]['Num point ACDC']
    numPt = '???' if pd.isnull(numPt) else int(numPt)
    heure = dfObsForm.iloc[0]['Heure début'].strftime('%H:%M')
    mc = folium.plugins.MarkerCluster(name='#{}/{} {}'.format(numPt, formId, heure),
                                      options=dict(maxClusterRadius=200, disableClusteringAtZoom=16,
                                                   spiderfyOnMaxZoom=False))
    for indObs, sObs in dfObsForm.iterrows():
        sObs = sObs.fillna('')
        eff = sObs['Estimation'] + str(sObs['Nombre']) + ('' if not sObs['Détails'] else ' ({})'.format(sObs['Détails']))
        ca = '' if not sObs['Code atlas'] else ' (c{})'.format(int(sObs['Code atlas']))
        mrk = folium.CircleMarker(location=(sObs['Lat (WGS84)'], sObs['Lon (WGS84)']), 
                                  radius=8, line_color='#ff0000', fill=True, fill_color='orange', fill_opacity=0.6,
                                  popup=folium.Popup('#{} {} {}-{} : {} {}{}' \
                                                     .format(numPt, heure, sObs['Date'].strftime('%d/%m/%Y'), 
                                                             'xx::yy', #sObs['Horaire'].strftime('%H:%M'),
                                                             eff, sObs['Nom espèce'], ca)))
        mrk.add_to(mc)
    mc.add_to(mp)
    
mp.fit_bounds(mp.get_bounds())
mp

In [ ]:
# Formulaires sans numéro de point
dfForms.loc[dfForms['Num point ACDC'].isnull()].drop(columns=['Num point ACDC'])

In [ ]:
dfForms['Num point ACDC'] = dfForms['Num point ACDC'].astype(int)
dfObs['Num point ACDC'] = dfObs['Num point ACDC'].astype(int)

# 6. Suppression des formulaires non conformes

In [ ]:
len(dfObs), len(dfForms)

In [ ]:
# Trop courts : Normalement, tous les formulaires ont duré au moins 10mn (à qq s près => >= 9 mn)
dfForms[dfForms['Durée'] < pd.Timedelta(minutes=9)]

In [ ]:
dfForms.drop(53130, inplace=True)
len(dfForms)

In [ ]:
dfObs.drop(dfObs[dfObs['ID liste'] == 53130].index, inplace=True)
len(dfObs)

# 7. Bilan formulaires / observateurs

In [ ]:
# Bilan par observateur et par point : listes des points attendus et réalisés, par passage
dfBilanReal = \
    dfPoints.reset_index()[['numero', 'naturalist',]] \
            .rename(columns=dict(numero='Num point ACDC', naturalist='Observateur')) \
        .join(dfForms[['Num point ACDC', 'Observateur', 'Passage']] \
              .set_index(['Num point ACDC', 'Observateur']),
              on=['Num point ACDC', 'Observateur'], how='outer')
len(dfBilanReal)

In [ ]:
# Rappel : les points non attribués (donc pas fait, normal)
list(dfBilanReal.loc[dfBilanReal.Observateur.isnull(), 'Num point ACDC'])

In [ ]:
dfBilanReal = dfBilanReal[dfBilanReal.Observateur.notnull()]
len(dfBilanReal)

In [ ]:
# Les points attribués et pas faits du tout
dfBilanReal[dfBilanReal.Observateur.notnull() & dfBilanReal.Passage.isnull()]

In [ ]:
# Bilan global par observateur : nb de points effectivement inventoriés à chaque passage
dfBilan = dfBilanReal.groupby(['Observateur', 'Passage']).count().rename(columns={'Num point ACDC':'Réalisés'}).unstack()

In [ ]:
# Bilan par observateur : nb de formulaires attendu et réalisé, par passage
dfBilan = dfBilanAttr.loc[dfForms.Observateur.unique(), ['Naturalist']] \
              .join(dfBilan).rename(columns={'Naturalist':'Attendus'})
dfBilan

In [ ]:
dfBilan.sum().to_dict()

In [ ]:
# Répartition des formulaires par durée (résolution : 1mn ; n minutes = dans [n et n+1[ minutes)
dfForms.groupby(['Durée', 'Passage']).size().unstack()

In [ ]:
# Répartition des données par minute d'inventaire (résolution : 1mn ; n minutes = dans [n et n+1[ minutes)
df = dfObs.groupby(['Minute', 'Passage']).size().unstack()
df

In [ ]:
# % de données au delà de la 9ème minute
100 * df[df.index >= pd.Timedelta(minutes=10)].sum() / df.sum()

# 7. Première mise en forme des données en vue de l'exploitation DS

* format des dates,
* code atlas absent => 0,
* extraction des nb de mâles adultes, d'autres adultes et de juvéniles (à partir nbre, code atlas et détails),
* séparation des 5 1ères minutes de chaque formulaire
* suppression des données "en vol"

In [ ]:
dfObs['Détails'].fillna(value='', inplace=True)
dfObs['Code atlas'].fillna(value=0, inplace=True)
dfObs.Date = dfObs.Date.apply(pd.Timestamp)

In [ ]:
dfObs['Détails'].unique()

In [ ]:
# Codes Atlas Biolovision "à 20 valeurs" (les code de Faune Auvergne sont un sous-ensemble)
KCAAucun   =  0 # Pas de code Atlas
KCAObsSimp =  1 # Observation de l'espèce pendant la période de nidification
KCAObsBiot =  2 # Observation de l'espèce pendant la période de nidification dans un biotope adéquat
KCAMalChan =  3 # Mâle chanteur présent en période de nidification, cris nuptiaux ou tambourinage entendus,
                # mâle vu en parade dans un habitat favorable
KCACoupPer =  4 # Couple pendant la période de nidification dans un biotope adéquat
KCACoupTer =  5 # Comportement territorial d'un couple (chant, querelles avec des voisins, etc.)
                # au moins 2 jours a plus d'une semaine d'intervalle dans le même territoire
KCACoupNup =  6 # Comportement nuptial (mâle et femelle observés)
KCAVisNidP =  7 # Visite d'un site de nidification probable
KCACrAlInq =  8 # Cri d'alarme ou de crainte des adultes ou autre comportement agité suggérant la présence
                # d'un nid ou de jeunes aux alentours
KCAPlaqInc =  9 # Plaque incubatrice d'une femelle capturée
KCAConstNd = 10 # Construction d'un nid ou forage d'une cavité
KCAIndSimu = 11 # Oiseau simulant une blessure ou détournant l'attention
KCANidUtil = 12 # Découverte d'un nid ayant été utilisé durant la période de nidification actuelle
KCAJuvDepd = 13 # Jeunes venant de s'envoler (nidicoles) ou poussins en duvet (nidifuges)
KCAAdSitNi = 14 # Adulte gagnant ou quittant un site de nid; comportement révélateur d'un nid occupé
                # dont le contenu ne peut être vérifié (trop haut ou dans une cavité) ou adulte incubant
KCATranFie = 15 # dulte transportant des fientes
KCATranNou = 16 # Adulte transportant de la nourriture pour les jeunes
KCACoqOeuf = 17 # Coquilles d'oeufs éclos (de la période de nidification actuelle)
KCAAdCouve = 18 # Nid avec adulte vu couvant
KCANiOeufs = 19 # Nid avec oeufs
KCAPousNid = 20 # Jeunes au nid vus ou entendus
KCANdfPoss = 30 # Nidification possible
KCANdfProb = 40 # Nidification probable
KCANdfCert = 50 # Nidification certaine
KCAAbsRech = 99 # Espèce absente malgré des recherches

In [ ]:
# Codes atlas et coefficients d'effectifs correspondants
dfCodesAtlas = pd.read_excel('ACDC/VisioNatureCodesAtlas.xlsx', index=0)[['Codes Biolovision', 'Texte FR']]
dfCodesAtlas.set_index('Codes Biolovision', inplace=True)
#dfCodesAtlas.drop([KCAAucun], inplace=True) # Parfois utilisé par erreur ... tant pis on garde
dfCodesAtlas.drop([KCANdfPoss, KCANdfProb, KCANdfCert, KCAAbsRech], inplace=True)
dfCodesAtlas['nMalAd'] = 0.0
dfCodesAtlas.loc[KCAMalChan, 'nMalAd'] = 1
dfCodesAtlas.loc[KCACoupPer, 'nMalAd'] = 0.5
dfCodesAtlas.loc[KCACoupTer, 'nMalAd'] = 0.5
dfCodesAtlas.loc[KCACoupNup, 'nMalAd'] = 0.5
dfCodesAtlas['nJuv'] = 0.0
dfCodesAtlas.loc[KCAJuvDepd, 'nJuv'] = 1
dfCodesAtlas.loc[KCANiOeufs, 'nJuv'] = 1
dfCodesAtlas.loc[KCAPousNid, 'nJuv'] = 1
dfCodesAtlas['nAutAd'] = 1
dfCodesAtlas.loc[KCAAucun, 'nAutAd'] = 1
dfCodesAtlas.loc[KCAMalChan, 'nAutAd'] = 0
dfCodesAtlas.loc[KCACoupPer, 'nAutAd'] = 0.5
dfCodesAtlas.loc[KCACoupTer, 'nAutAd'] = 0.5
dfCodesAtlas.loc[KCACoupNup, 'nAutAd'] = 0.5
dfCodesAtlas.loc[KCAJuvDepd, 'nAutAd'] = 0
dfCodesAtlas.loc[KCANiOeufs, 'nAutAd'] = 0
dfCodesAtlas.loc[KCAPousNid, 'nAutAd'] = 0

assert all(dfCodesAtlas[['nMalAd', 'nAutAd', 'nJuv']].sum(axis='columns') == 1)

dfCodesAtlas

In [ ]:
# Effectif détaillé, par sexe et âge (ad ou juv=pul+1ère année),
# à partir des colonnes ['Nombre', 'Détails', 'Code atlas'],
# supposée formatée ainsi : "<n1>x <sexe> <age> <condition>, ..., <np>x <sexe> <age> <condition>"
# Attention: 
# * Les individus 'en vol' ne sont pas comptés.
# * On conserve quand même les données sans code atlas : certains observateurs les ont beaucoup oubliés !?

KDetCols = ['nMalAd', 'nAutAd', 'nJuv']

KReNumExpect = re.compile(r"([0-9]+)x(.*)")

KEffInvd = pd.Series({ col : np.nan for col in KDetCols })

def calculerEffectifs(sObs, verbose=False):

    # Détails et nbres associés
    effDets = { col : 0 for col in KDetCols }
    
    lstDets = sObs['Détails'].split("/")
    vol = False
    for rawDet in lstDets:
        
        rawDet = rawDet.strip()
        if not rawDet:
            continue
        
        mo = KReNumExpect.match(rawDet)
        if mo:
            
            num = int(mo.group(1))
            sxAgCn = mo.group(2).strip()
            
            sx = 'mal' if 'mâle' in sxAgCn else 'fem' if 'femelle' in sxAgCn else 'alt'
            ag = 'juv' if '1ère année' in sxAgCn or 'poussin' in sxAgCn or 'immature' in sxAgCn \
                       else 'adu' if 'adulte' in sxAgCn or 'année' in sxAgCn or '1 an' in sxAgCn else 'alt'
            cn = 'vol' if 'vol' in sxAgCn else 'alt'
            
            if cn != 'vol':
                
                cat = 'nJuv' if ag == 'juv' else 'nMalAd' if sx == 'mal' else 'nAutAd'
            
                effDets[cat] += num
                
            else:
                
                if len(lstDets) == 1:
                    return KEffInvd
                vol = True
            
        else:
            
            print("Donnée ignorée : Colonne Détails malformée : '{}'".format(rawDet))
            if verbose:
                print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
            
            return KEffInvd
    
    # Code Atlas et Nombre total associé
    codAtls = int(sObs['Code atlas'])
    if codAtls != KCAAucun and codAtls not in dfCodesAtlas.index and not vol:
        print("Donnée ignorée : Code Atlas {} sans intérêt, mais pas en vol : détails={}" \
              .format(codAtls, sObs['Détails']))
        if verbose:
            print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
        return KEffInvd

    nbre = sObs['Nombre']
    if codAtls in [KCACoupPer, KCACoupTer, KCACoupNup] and nbre < 2:
        print("Donnée suspecte : Code Atlas pour les couples : {}, mais nb individus < 2 : {} !" \
              .format(codAtls, nbre))
        if verbose:
            print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
    
    effNbCode = { colCoefMult : dfCodesAtlas.loc[codAtls, colCoefMult] * nbre \
                 for colCoefMult in KDetCols }
    
    # Quand code atlas indique couple, et que nbre impair d'individus => autre adulte par défaut
    demiMal = effNbCode['nMalAd'] - int(effNbCode['nMalAd'])
    effNbCode['nMalAd'] -= demiMal
    effNbCode['nAutAd'] += demiMal
    
    # Choix final : Priorité aux détails si présents
    if any(effDets.values()):
        if sum(effDets.values()) != sum(effNbCode.values()):
            print("Donnée ignorée : Détails {} et Nombre {} * CodeAtlas {} incohérents : '{}' / '{}'" \
                  .format(rawDet, nbre, codAtls, effDets, effNbCode))
            if verbose:
                print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
            return KEffInvd
        eff = effDets
    else:
        eff = effNbCode
        
    return pd.Series(eff)

In [ ]:
if len(dfObs.Estimation.dropna().unique()) > 0:
    print('Attention : On prend les estimations pour des valeurs exactes.')

In [ ]:
obsCols = ['ID liste', 'Passage', 'Num point ACDC', 'Observateur', 'Date', 'Horaire',
           'Nom espèce', 'Lieu-dit', 'Commune', 'Estimation', 'Nombre', 'Détails', 'Code atlas']

In [ ]:
dfObs[KDetCols] = dfObs[obsCols].apply(calculerEffectifs, axis='columns', verbose=True)
dfObs[KDetCols] = dfObs[KDetCols].fillna(0).astype(int)
dfObs['nDetTot'] = dfObs[KDetCols].sum(axis=1).astype(int)

In [ ]:
# Suppression des données "en vol"
df = dfObs[dfObs['Détails'].str.contains('vol', case=False)]
len(dfObs), len(df) # df

In [ ]:
dfObs.drop(df.index, inplace=True)
len(dfObs)

In [ ]:
#dfObs[dfObs['Code atlas'].isin([4, 5]) & (dfObs.Nombre == 1)].to_excel('ACDC/ACDC2019b-CoupleMais1SeulInd.xlsx', index=False)

In [ ]:
# Extraction des 5 1ères minutes
KDetColsTot = KDetCols + ['nDetTot']

KDetColsTot10 = [c+'10' for c in KDetColsTot]

dfObs.rename(columns=dict(zip(KDetColsTot, KDetColsTot10)), inplace=True)

In [ ]:
KDetColsTot5 = [c+'5' for c in KDetColsTot]
for col in KDetColsTot:
    dfObs[col+'5'] = dfObs.loc[dfObs['Minute'] < pd.Timedelta(minutes=5), col+'10']
    dfObs[col+'5'] = dfObs[col+'5'].fillna(0).astype(int)

In [ ]:
dfObs.sort_values(by=['Num point ACDC', 'Horaire']).head(20)

In [ ]:
# Vérifs
d = dict(donneesAvant5mn=len(dfObs[dfObs['Minute'] < pd.Timedelta(minutes=5)]),
         donneesApres5mn=len(dfObs[dfObs[KDetColsTot5].sum(axis='columns') == 0]),
         total=len(dfObs))
assert d['total'] == d['donneesAvant5mn'] + d['donneesApres5mn']
d

# 6. Premiers contrôles de cohérence des données

In [ ]:
# Cas des absences de code atlas 0 (KCAAucun) hors 'en vol' : des fois justifié, souvent un oubli ?
# (on les a comptés comme code atlas 2)
df = dfObs.loc[(dfObs['Code atlas'] == KCAAucun) & ~dfObs['Détails'].str.contains('vol'), obsCols]
#df.to_excel('ACDC/Codes0.xlsx', index=False)
df

In [ ]:
# Code atlas innatendu dans ce genre d'enquête
dfObs.loc[dfObs['Code atlas'].isin([KCAPlaqInc]), obsCols]

In [ ]:
# Comparaison nb individu total et somme des détails : différence = en vol (mais supprimés => 0)
d = dict(total=dfObs['Nombre'].sum(), detail=dfObs['nDetTot10'].sum(), 
         difference=dfObs['Nombre'].sum() - dfObs['nDetTot10'].sum(),
         en_vol=dfObs.loc[dfObs['Détails'].str.contains('vol'), 'Nombre'].sum())
assert d['difference'] == d['en_vol'], "Incohérence nbre / détails pour les non 'en vol'"
d

In [ ]:
dfObs[['Ref']+obsCols+KDetColsTot5+KDetColsTot10].sort_values(by=['Passage', 'Horaire', 'Num point ACDC']) \
    .to_excel('ACDC/ACDC2019-Naturalist.xlsx', index=False)

In [ ]:
dfObs.columns

# 7. Quelques stats

In [ ]:
# Stats données brutes tous formulaires.
{ 'NbObservateurs' : dfObs.Observateur.nunique(), 
  'NbFormulaires' : dfObs['ID liste'].nunique(),
  'NbJours' : dfObs.Date.nunique(),
  'NbDonnées' : len(dfObs),
  'NbIndividus' : dfObs.Nombre.sum() }

In [ ]:
dfObs['Minute'] = dfObs['Minute'].divide(pd.Timedelta(minutes=1)).astype(int)

## a. Fonction du temps lors des inventaires

In [ ]:
# Effectifs contactés par tranche de temps (5 / 10mn)
dfBilanTemps510 = dfObs[['Num point ACDC', 'Passage', 'nMalAd5', 'nDetTot5', 'nMalAd10', 'nDetTot10']] \
                     .groupby(['Num point ACDC', 'Passage']).sum()
dfBilanTemps510 = \
   dfBilanTemps510.join(dfObs[['Num point ACDC', 'Passage', 'nDetTot10']].rename(columns={'nDetTot10': 'nDonnées10'}) \
                  .groupby(['Num point ACDC', 'Passage']).count())
dfBilanTemps510 = \
   dfBilanTemps510.join(dfObs.loc[dfObs.nDetTot5 > 0, ['Num point ACDC', 'Passage', 'nDetTot5']].rename(columns={'nDetTot5': 'nDonnées5'}) \
                  .groupby(['Num point ACDC', 'Passage']).count())
dfBilanTemps510 = dfBilanTemps510.groupby('Passage').agg(['sum', 'mean'])
#dfBilanTemps510.columns = dfBilanTemps510.columns.swaplevel(0, 1)
dfBilanTemps510 #.sort_values(by=('a', 'nMalAd10'), ascending=False)

In [ ]:
# Effectifs contactés par tranche d'1 minute 
dfBilanTempsDet = dfObs[['Passage', 'Minute', 'nDetTot10']].groupby(['Minute', 'Passage']).sum().unstack()
dfBilanTempsDet.columns = dfBilanTempsDet.columns.levels[1]
_ = dfBilanTempsDet.plot(figsize=(16, 3), marker='.', grid=True,
                         title='Nb total d\'individus contactés par minute (en moyenne, par passage)')

In [ ]:
# La même chose ...
_ = dfBilanTempsDet[['a', 'b']].plot(figsize=(16, 4), kind='bar', stacked=False, grid=True,
                                     title='Nb total d\'individus contactés par minute (en moyenne, par passage)')

In [ ]:
# Effectifs cumulés contactés par tranche d'1 minute 
for pas in dfBilanTempsDet.columns:
    dfBilanTempsDet['cum({})%'.format(pas)] = 100 * dfBilanTempsDet[pas].cumsum() / dfBilanTempsDet[pas].sum()
_ = dfBilanTempsDet[[c for c in dfBilanTempsDet.columns if 'cum' in c]].plot(figsize=(16, 3), marker='.', grid=True,
                    title='%age cumulé du nb total d\'individus contactés par minute (en moyenne, par passage)')

In [ ]:
_ = dfBilanTempsDet[[c for c in dfBilanTempsDet.columns if 'cum' in c]].plot(figsize=(16, 3), grid=True,
                                                                             kind='bar', stacked=False,
                    title='%age cumulé du nb total d\'individus contactés par minute (en moyenne, par passage)')

In [ ]:
dfBilanTempsDet

## c. Fonction du temps au cours des matinées

In [ ]:
# Chargement des éphémérides solaires (UTC, source https://promenade.imcce.fr/fr/pages6/746.html)
dfEphem = pd.read_csv('SoleilEphemerides2019.txt', sep='\t', header=1, index_col=0)
dfEphem['2019-05-27':'2019-06-03']

In [ ]:
dfEphem = dfEphem.apply(lambda day: pd.Series({ hTyp : pd.NaT if pd.isnull(hVal) else pd.Timestamp(day.name + ' ' + hVal) \
                                               for hTyp, hVal in day.iteritems() }), axis='columns')
dfEphem.head()

In [ ]:
dfEphem.index = dfEphem.index.map(pd.Timestamp)
dfEphem.head()

In [ ]:
dfObs.head()

In [ ]:
# Calcul de l'HoraireUTC (en tenant compte des heures d'été / d'hiver ... bof, changement fin mars)
dfObs['HoraireUtc'] = dfObs.Horaire.apply(lambda ts: ts.tz_localize('Europe/Paris').tz_convert(None))
dfObs.loc[(dfObs.Date >= '2019-05-25'), ['Horaire', 'HoraireUtc']].head()

In [ ]:
# Ajout de l'heure de l'aube civile dans les données
dfObs = dfObs.join(dfEphem[['AubeCivil']], on='Date')
dfObs.head()

In [ ]:
dfObs.drop('AubeCivil', inplace=True, axis='columns')

In [ ]:
# Calcul du temps passé depuis l'aube civile du jour
dfObs = dfObs.join(dfEphem[['AubeCivil']], on='Date')

dfObs['DeltaAubeCivile'] = dfObs.HoraireUtc - dfObs.AubeCivil
dfObs['DeltaHAubeCivile'] = dfObs.DeltaAubeCivile.apply(lambda td: td.floor(freq='h'))

dfObs.loc[(dfObs.Date >= '2019-05-25'), ['HoraireUtc', 'AubeCivil', 'DeltaAubeCivile', 'DeltaHAubeCivile']].head()

In [ ]:
df.index.name = 'Heures depuis l\'aube'

In [ ]:
# Nbre de piafs total par heure "biologique" par observateur
df = dfObs[['Observateur', 'DeltaAubeCivile', 'nDetTot10']].groupby(['DeltaAubeCivile', 'Observateur']).sum() \
        .unstack().fillna(0).astype(int)
df.columns = df.columns.levels[1]
df.index.name = 'Heures depuis l\'aube'
df.index= df.index.map(lambda td: td.total_seconds()/60)
##df['Equipe'] = df.sum(axis='columns')
#df.to_excel('ACDC2019-ContactsIndividusParHeureDepuisAube.xlsx')

In [ ]:
(100 * df.sum(axis='columns') / df.sum().sum()) \
.plot(kind='bar', figsize=(16, 4), width=0.9, grid=True,
      title='ACDC 2019 : % global de contacts par minute depuis l\'aube, tous observateurs')

plt.savefig('ACDC/ACDC2019-ContactsParMinuteDepuisAube-pctGlobalTousObseurs.png', transparent=False)

In [ ]:
# Nbre de piafs total par heure "biologique" par observateur
df = dfObs[['Observateur', 'DeltaHAubeCivile', 'nDetTot10']].groupby(['DeltaHAubeCivile', 'Observateur']).sum() \
        .unstack().fillna(0).astype(int)
df.columns = df.columns.levels[1]
df.index.name = 'Heures depuis l\'aube'
df.index= df.index.map(lambda td: td.total_seconds()/3600)
#df['Equipe'] = df.sum(axis='columns')
df.to_excel('ACDC2019-ContactsIndividusParHeureDepuisAube.xlsx')

In [ ]:
(100 * df.sum(axis='columns') / df.sum().sum()) \
.plot(kind='bar', figsize=(16, 4), width=0.9, grid=True,
      title='ACDC 2019 : % global de contacts par heure depuis l\'aube, tous observateurs')

plt.savefig('ACDC/ACDC2019-ContactsParHeureDepuisAube-pctGlobalTousObseurs.png', transparent=False)

In [ ]:
df

In [ ]:
(100 * df / df.sum().sum()).plot(kind='bar', figsize=(16, 4), width=0.9, grid=True,
                                 title='ACDC 2019 : % global de contacts par heure depuis l\'aube')

plt.savefig('ACDC/ACDC2019-ContactsParHeureDepuisAube-pctGlobal.png', transparent=False)

In [ ]:
(100 * df / df.sum()).plot(kind='bar', figsize=(16, 4), width=0.9, grid=True,
                           title='ACDC 2019 : % personnel de contacts par heure depuis l\'aube')

plt.savefig('ACDC/ACDC2019-ContactsParHeureDepuisAube-pctPerso.png', transparent=False)

In [ ]:
# Version + rapide, sans tenir compte de l'heure réelle par rapport au soleil

In [ ]:
dfObs['Heure'] = dfObs.Horaire.apply(lambda ts: ts.floor(freq='h').strftime('%H:%M'))

In [ ]:
df = dfObs[['Observateur', 'Heure', 'nDetTot10']].groupby(['Heure', 'Observateur']).sum().unstack().fillna(0).astype(int)
df.columns = df.columns.levels[1]
#df['Equipe'] = df.sum(axis='columns')
df #.to_excel('ACDC2019-ContactsIndividusParHeure.xlsx')

In [ ]:
df.plot(kind='bar', figsize=(18, 4), title='ACDC 2019 : Nbre de contacts individuels par heure du jour')

In [ ]:
# Les points par heure depuis l'aube

In [ ]:
dfObs['HeureDebutUtc'] = dfObs['Heure début'].apply(lambda ts: ts.tz_localize('Europe/Paris').tz_convert(None))
dfObs['DeltaFormAubeCivile'] = dfObs.HeureDebutUtc - dfObs.AubeCivil
dfObs['DeltaFormHAubeCivile'] = dfObs.DeltaFormAubeCivile.apply(lambda td: td.floor(freq='h'))

In [ ]:
# Nbre de piafs total par passage, point et minute "biologique" (tous observateur)
df = dfObs[['Passage', 'Observateur', 'Num point ACDC', 'DeltaFormAubeCivile', 'nDetTot10']] \
       .groupby(['Passage', 'Observateur', 'Num point ACDC', 'DeltaFormAubeCivile']).sum()
df.reset_index(inplace=True)
df.drop('Num point ACDC', axis='columns', inplace=True)
df.DeltaFormAubeCivile = df.DeltaFormAubeCivile.apply(lambda td: td.total_seconds()/3600)
df.set_index('Passage', inplace=True)
df.head()

In [ ]:
# Les points (par passage) selon leur nbre de piaf total et leur heure+minute biologique de début
KObseurColor = dict(zip(dfForms.Observateur.unique(), plt.cm.tab10.colors))

fig = plt.figure(figsize=(16, 8))
fig.suptitle('ACDC 2019 : Points 10 mn : Nb total de contacts fonction de l\'heure précise de début depuis l\'aube' \
             ' (1 couleur par observateur)',
             fontsize=18, y=1.04)
gs = pltg.GridSpec(nrows=2, ncols=1)

spInd = 0
for pas in df.index.unique():
    
    axes = fig.add_subplot(gs[spInd])
    df2Plot = df.loc[pas, ['DeltaFormAubeCivile', 'nDetTot10', 'Observateur']]
    df2Plot['CoulObseur'] = df2Plot.Observateur.apply(KObseurColor.get)
    df2Plot.plot(ax=axes, kind='scatter', grid=True,
                 x='DeltaFormAubeCivile', y='nDetTot10',
                 color=df2Plot.CoulObseur, s=50, alpha=0.8)
    axes.set_title('Passage {}'.format(pas), fontsize=16)
    axes.set_xlim(left=0, right=7)
    axes.set_xlabel('Heures depuis l\'aube civile', fontsize=12)
    axes.set_ylabel('Nb total d\'individus', fontsize=12)
    #axes.legend(['Nb total d\'individus'], fontsize=12) #, bbox_to_anchor=(1.02, 1), loc=2)
    
    spInd += 1
    
fig.tight_layout()

plt.savefig('ACDC/ACDC2019-Points-NbContactsParHeureDepuisAube.png', transparent=False, bbox_inches='tight')

In [ ]:
# Essai représentation densité : bof ...
_ = plt.hexbin(x=df2Plot.DeltaFormAubeCivile, y=df2Plot.nDetTot10, gridsize=(5, 8))

In [ ]:
df = dfObs[['Passage', 'Num point ACDC', 'DeltaHAubeCivile', 'nDetTot10']] \
       .groupby(['Passage', 'DeltaHAubeCivile']).agg({ 'nDetTot10': sum, 'Num point ACDC': 'nunique'})
df.rename(columns={'Num point ACDC': 'nbPoints' }, inplace=True)

df.loc[('b', pd.Timedelta(hours=0)), ['nDetTot10', 'nbPoints']] = (0, 0.1) # Fix missing first hour (Matplotlib bug ?)

df.sort_index(inplace=True)

df.reset_index(inplace=True)

df.DeltaHAubeCivile = df.DeltaHAubeCivile.apply(lambda td: td.total_seconds()/3600).astype(int)

df['nDetTot10ParPoint'] = df.nDetTot10 / df.nbPoints

df.set_index('Passage', inplace=True)

df

In [ ]:
# Le nbre total moyen de piafs par heure biologique de contact
fig = plt.figure(figsize=(16, 8))
fig.suptitle('ACDC 2019 : Nb moyen de contacts par point (10 mn), par heure depuis l\'aube, tous observateurs',
             fontsize=18, y=1.04)
gs = pltg.GridSpec(nrows=2, ncols=1)

spInd = 0
for pas in df.index.unique():
    
    axes = fig.add_subplot(gs[spInd])
    df2Plot = df.loc[pas].set_index('DeltaHAubeCivile', drop=True)
    df2Plot.nDetTot10ParPoint.plot(ax=axes, kind='bar', width=0.8, grid=True)
    for h, s in df2Plot.iterrows():
        if s.nbPoints >= 1:
            axes.text(h, 0.5, "{:.0f} pts".format(s.nbPoints), 
                      color='white' if s.nDetTot10ParPoint > 0.5 else 'black',
                      fontsize=14, fontweight='bold', ha='center')
            axes.text(h, s.nDetTot10ParPoint-1.3, "{:.0f} inds".format(s.nDetTot10), 
                      color='white',
                      fontsize=14, fontweight='bold', ha='center')
    axes.set_title('Passage {}'.format(pas), fontsize=16)
    axes.set_xlim(left=-0.5, right=7.5)
    axes.set_xlabel('Heures depuis l\'aube civile', fontsize=12)
    axes.set_ylabel('Nb moyen d\'individus par point', fontsize=12)
    #axes.legend(['Nb moyen d\'individus par point']) #, bbox_to_anchor=(1.02, 1), loc=2)
    
    spInd += 1
    
fig.tight_layout()

plt.savefig('ACDC/ACDC2019-NbMoyContactsParPointParHeureDepuisAube.png', transparent=False, bbox_inches='tight')

## c. Fonction de l'observateur

In [ ]:
# Détail par observateur et par passage.
# TODO : 1 graphique par observateur

In [ ]:
# Nbre d'individus contactés par passage et par point, pour les 2 tranches de temps
dfBilanObseursPointDet = dfObs[['Observateur', 'Num point ACDC', 'Passage',
                                'nMalAd5', 'nAutAd5', 'nDetTot5', 'nMalAd10', 'nAutAd10', 'nDetTot10']] \
                           .groupby(['Observateur', 'Num point ACDC', 'Passage']).sum().unstack()
dfBilanObseursPointDet.columns = dfBilanObseursPointDet.columns.swaplevel(0, 1)
dfBilanObseursPointDet.sort_index(axis='columns', inplace=True)
dfBilanObseursPointDet

In [ ]:
dfBilanTempsObseurs = \
   dfObs[['Passage', 'Observateur', 'Minute', 'nMalAd10', 'nAutAd10', 'nDetTot10']] \
     .groupby(['Minute', 'Observateur', 'Passage']).sum().unstack()
dfBilanTempsObseurs.columns = dfBilanTempsObseurs.columns.swaplevel(0, 1)
passages = dfBilanTempsObseurs.columns.levels[0]
for col in dfBilanTempsObseurs.columns.levels[1]:
    dfBilanTempsObseurs[('total', col)] = dfBilanTempsObseurs[[(pas, col) for pas in passages]].sum(axis='columns')
dfBilanTempsObseurs.sort_index(axis='columns', inplace=True)
dfBilanTempsObseurs

In [ ]:
dfBilanTempsObseurs.head()

In [ ]:
# Nbre de données par point par observateur par minute
dfNbPts = pd.DataFrame(index=dfBilanTempsObseurs.index)
dfNbPts.reset_index(inplace=True)
dfNbPts = dfNbPts.join(dfBilan, on='Observateur')
dfNbPts[('Réalisés', 'total')] = sum(dfNbPts[('Réalisés', pas)] for pas in dfBilanTempsObseurs.columns.levels[0][:-1])
dfNbPts.set_index(['Minute', 'Observateur'], inplace=True)

dfBilanTempsObseursMoyParPt = dfBilanTempsObseurs.copy()

for pas in dfBilanTempsObseursMoyParPt.columns.levels[0]:
    for col in dfBilanTempsObseursMoyParPt.columns.levels[1]:
        dfBilanTempsObseursMoyParPt[(pas, col)] = dfBilanTempsObseursMoyParPt[(pas, col)] / dfNbPts[('Réalisés', pas)]
            
dfBilanTempsObseursMoyParPt.sort_values(by=('total', 'nDetTot10'), ascending=False, inplace=True)
dfBilanTempsObseursMoyParPt

In [ ]:
df = dfBilanTempsObseursMoyParPt[('a', 'nDetTot10')].unstack()
df.rename(columns=dict(zip(df.columns, ['Observateur '+chr(ord('A')+i) for i in range(len(df.columns))])), inplace=True)
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='Passage A : Nb d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = 100 * df.cumsum() / df.sum()
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='Passage A : % cumulé du total d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = dfBilanTempsObseursMoyParPt[('b', 'nDetTot10')].unstack()
df.rename(columns=dict(zip(df.columns, ['Observateur '+chr(ord('A')+i) for i in range(len(df.columns))])), inplace=True)
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='Passage B : Nb d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = 100 * df.cumsum() / df.sum()
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='Passage B : % cumulé du total d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = dfBilanTempsObseursMoyParPt[('total', 'nDetTot10')].unstack()
df.rename(columns=dict(zip(df.columns, ['Observateur '+chr(ord('A')+i) for i in range(len(df.columns))])), inplace=True)
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='2019 (2 passages) : Nb d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
df = 100 * df.cumsum() / df.sum()
_ = df.plot(figsize=(16, 5), marker='.', grid=True,
            title='2019 (2 passages) : % cumulé du total d\'individus contactés par minute (moyenne par observateur)')

In [ ]:
# Rappel nb de points par passage et par observateur : attendu et effectué.
dfBilan

In [ ]:
# Nb de points et de données par observateur
dfBilanObseurs = dfBilan.join(dfObs[['Observateur', 'Horaire', 'Passage']].groupby(['Observateur', 'Passage']) \
                                .count().unstack())
dfBilanObseurs = dfBilanObseurs.join(dfObs[['Observateur', 'nDetTot10', 'nDetTot5', 'nMalAd10', 'nMalAd5', 'nAutAd10', 'nAutAd5', 'Passage']] \
                                     .groupby(['Observateur', 'Passage']).sum().unstack())
dfBilanObseurs.columns = pd.MultiIndex.from_tuples([t if isinstance(t, tuple) else (t, 'total') for t in dfBilanObseurs.columns])
dfBilanObseurs.rename(columns={'Horaire': 'nDonnées'}, level=0, inplace=True)
dfBilanObseurs.columns = dfBilanObseurs.columns.swaplevel(0, 1)

passages = dfBilanObseurs.columns.levels[0][:-1]
for col in dfBilanObseurs.columns.levels[1]:
    if col not in ['Attendus', 'Réalisés']:
        dfBilanObseurs[('total', col)] = dfBilanObseurs[[(pas, col) for pas in passages]].sum(axis='columns')

dfBilanObseurs.sort_index(axis='columns', inplace=True)
dfBilanObseurs.sort_values(by=('total', 'nDetTot10'), ascending=False, inplace=True)
dfBilanObseurs

In [ ]:
# Nbre de données par point par observateur
dfBilanObseursMoyParPt = dfBilanObseurs.copy()

for pas in dfBilanObseursMoyParPt.columns.levels[0]:
    nPtsCol = 'Attendus' if pas == 'total' else 'Réalisés'
    for col in dfBilanObseursMoyParPt.columns.levels[1]:
        if col not in ['Attendus', 'Réalisés']:
            dfBilanObseursMoyParPt[(pas, col)] = dfBilanObseursMoyParPt[(pas, col)] / dfBilanObseursMoyParPt[(pas, nPtsCol)]
            
dfBilanObseursMoyParPt.sort_values(by=('total', 'nDetTot10'), ascending=False, inplace=True)
dfBilanObseursMoyParPt

## c. Fonction de l'espèce

In [ ]:
# Nb de données / individus par espèce.
dfBilanEspeces = dfObs[['Nom espèce', 'nDetTot10', 'nDetTot5', 'nMalAd10', 'nMalAd5', 'nAutAd10', 'nAutAd5', 'Passage']] \
                .groupby(['Nom espèce', 'Passage']).sum().unstack()
dfBilanEspeces.columns = dfBilanEspeces.columns.swaplevel(0, 1)
dfBilanEspeces.sort_index(axis='columns', inplace=True)

passages = dfBilanEspeces.columns.levels[0]
for col in dfBilanEspeces.columns.levels[1]:
    dfBilanEspeces[('total', col)] = dfBilanEspeces[[(pas, col) for pas in passages]].sum(axis='columns')

dfBilanEspeces.sort_values(by=('total', 'nDetTot10'), ascending=False, inplace=True)
dfBilanEspeces

In [ ]:
# Export Excel
xlsWriter = pd.ExcelWriter('ACDC/ACDC2019-Bilan.xlsx')
dfBilanObseurs.to_excel(xlsWriter, sheet_name='Obseurs')
dfBilanObseursMoyParPt.to_excel(xlsWriter, sheet_name='ObseursParPt')
dfBilanEspeces.to_excel(xlsWriter, sheet_name='Espèces')
dfBilanTemps510.to_excel(xlsWriter, sheet_name='Temps510')
dfBilanTempsDet.to_excel(xlsWriter, sheet_name='TempsDétails')
dfBilanTempsObseurs.to_excel(xlsWriter, sheet_name='TempsObseurs')
dfBilanTempsObseursMoyParPt.to_excel(xlsWriter, sheet_name='TempsObseursParPt')
xlsWriter.save()
xlsWriter.close()

# Bac à sable

In [ ]:
# Codes Atlas, par copier + coller de https://wiki.biolovision.net/index.php?title=Correspondance_codes_atlas
dfCodesAtlas = pd.read_csv(io.StringIO("""Codes Biolovision	Codes utilisés en Suisse, Italie et partiellement en France	Codes utilisés en Allemagne et Catalogne	Codes utilisés partiellement en France (EBCC)	Codes utilisés en Autriche	Codes utilisés en Pologne	Texte FR	Texte EN
0	-	-	-	-	-	Pas de code atlas	No atlas code
1	1					Observation de l'espèce pendant la période de nidification	Species observed in breeding season
2	2	A1	1	H	O	Observation de l'espèce pendant la période de nidification dans un biotope adéquat	Species observed in breeding season in possible nesting habitat
3	3	A2	2	S	S	Mâle chanteur présent en période de nidification, cris nuptiaux ou tambourinage entendus, mâle vu en parade dans un habitat favorable	Singing, drumming or displaying male present in breeding season in possible nesting habitat
4	4	B3	3	P	PR	Couple pendant la période de nidification dans un biotope adéquat	Pair (male and female) within safe dates, and in suitable breeding habitat
5	5	B4	4	T	TE	Comportement territorial d'un couple (chant, querelles avec des voisins, etc.) au moins 2 jours a plus d'une semaine d'intervalle dans le même territoire	Territorial behaviour (song, fights with neighbour etc.) on at least two different days a week or more apart at same place indicating a permanently occupied territory
6	6	B5	5	D	KT	Comportement nuptial (mâle et femelle observés)	Courtship behavior (aerial displays, courtship feeding) or copulation
7	7	B6	6	N	OM	Visite d'un site de nidification probable	Visiting probable nest site
8	8	B7	7	A	NP	Cri d'alarme ou de crainte des adultes ou autre comportement agité suggérant la présence d'un nid ou de jeunes aux alentours	Agitated behavior and/or anxiety calls from an adult, suggesting presence of nearby nest or young
9	9	B8	8	I	PL	Plaque incubatrice d'une femelle capturée	Brood patch (Note: code only applies to birds observed in hand and is reserved for experienced birder only)
10	10	B9	9	B	BU	Construction d'un nid ou forage d'une cavité	Nest building observed at nest site (Note: for nest building by wrens, woodpeckers, kingfisher...)
11	11	C10	10	DD	UDA	Oiseau simulant une blessure ou détournant l'attention	Distraction display (especially injury feigning, such as broken wing display) or attacking/dive-bombing humans in defense of unobserved nest or young
12	12	C11a	11	UN	GNS	Découverte d'un nid ayant été utilisé durant la période de nidification actuelle	Used nest (occupied within period of survey); includes inactive nests
13	13	C12	12	FL	MŁO	Jeunes venant de s'envoler (nidicoles) ou poussins en duvet (nidifuges)	Recently fledged young that are incapable of sustained flight
14	14	C13a	13	ON	ZAJ	Adulte gagnant ou quittant un site de nid; comportement révélateur d'un nid occupé dont le contenu ne peut être vérifié (trop haut ou dans une cavité) ou adulte incubant	Occupied nest, but contents not observed; adults entering and remaining for a period of time, then leaving or exchanging duties
15	15	C14a				Adulte transportant des fientes	Adult carrying a fecal sac
16	16	C14b	14	FY	POD	Adulte transportant de la nourriture pour les jeunes	Adult carrying food for young
17	17	C11b		
Coquilles d'oeufs éclos (de la période de nidification actuelle)	Eggshells found (laid within period of survey)
18	18	C13b			WYS	Nid avec adulte vu couvant	Nest with adult incubating
19	19	C15	15	NE	JAJ	Nid avec oeufs	Nest containing eggs
20		C16	16	NY	PIS	Jeunes au nid vus ou entendus	Nest with young seen or heard.
30	30	A	30	30	A	Nidification possible	Possible breeding
40	40	B	40	40	B	Nidification probable	Probable breeding
50	50	C	50	50	C	Nidification certaine	Confirmed breeding
99	99	E99	99	99	NOBS	Espèce absente malgré des recherches	Not observed despite active search
19 codes	20 codes	16 codes	16 codes	17 codes	
"""), sep='\t')

In [ ]:
# Debug extraction nbre / sexe des détails, nbre, code atlas

In [ ]:
verbose = False
sObs = dfObs.loc[2907]
print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))

In [ ]:
print(sObs['Détails'])

# Détails
effDets = { col : 0 for col in KDetCols }

lstDets = sObs['Détails'].split("/")
vol = False
for rawDet in lstDets:

    rawDet = rawDet.strip()
    if not rawDet:
        continue

    mo = KReNumExpect.match(rawDet)
    if mo:

        num = int(mo.group(1))
        sxAgCn = mo.group(2).strip()

        sx = 'mal' if 'mâle' in sxAgCn else 'fem' if 'femelle' in sxAgCn else 'alt'
        ag = 'juv' if '1ère année' in sxAgCn or 'poussin' in sxAgCn or 'immature' in sxAgCn \
                   else 'adu' if 'adulte' in sxAgCn or 'année' in sxAgCn or '1 an' in sxAgCn else 'alt'
        cn = 'vol' if 'vol' in sxAgCn else 'alt'

        if cn != 'vol':

            cat = 'nJuv' if ag == 'juv' else 'nMalAd' if sx == 'mal' else 'nAutAd'

            effDets[cat] += num

        else:

            if len(lstDets) == 1:
                break
            vol = True

    else:

        print("Attention, donnée ignorée : Colonne Détails malformée : '{}'".format(rawDet))
        
vol, effDets, num, sx, ag, cn

In [ ]:
# Code Atlas et Nombre total associé
codAtls = int(sObs['Code atlas'])
if codAtls != 0 and codAtls not in dfCodesAtlas.index and not vol:
    print("Attention, donnée ignorée : Code Atlas {} sans intérêt, mais pas en vol d={}, n={}" \
          .format(codAtls, sObs['Détails'], sObs['Nombre']))
    if verbose:
        print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
    return KEffInvd

nbre = sObs['Nombre']
effNbCode = { colCoefMult : dfCodesAtlas.loc[codAtls, colCoefMult] * nbre \
             for colCoefMult in KDetCols }
if any(effDets.values()):
    if sum(effDets.values()) != sum(effNbCode.values()):
        print("Attention, donnée ignorée : Détails {} et Nombre {} * CodeAtlas {} incohérents : '{}' / '{}'" \
              .format(rawDet, nbre, codAtls, effDets, effNbCode))
        if verbose:
            print(', '.join(['{}:{}'.format(k[:3], v) for k, v in sObs.iteritems()]))
        return KEffInvd
    eff = effDets
else:
    eff = effNbCode

In [ ]:
df = pd.read_excel('ACDC/ACDC2019a-ExportClementRollant.xlsx')
df

In [ ]:
df['Code atlas'].unique()